In [1]:
import os
import re
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
from keras import models, layers

2023-09-15 09:01:04.119609: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-15 09:01:04.154427: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-15 09:01:04.155359: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-15 09:01:04.758750: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
num_threads = 5
os.environ["OMP_NUM_THREADS"] = "5"
os.environ["TF_NUM_INTRAOP_THREADS"] = "5"
os.environ["TF_NUM_INTEROP_THREADS"] = "5"
tf.config.threading.set_inter_op_parallelism_threads(num_threads)
tf.config.threading.set_intra_op_parallelism_threads(num_threads)
tf.config.set_soft_device_placement(True)

In [3]:
MAX_WORDS_IN_SENTENCE = 500
WORD2VEC_DIMENSIONS = 100
CSV_PATH = './data/reviews-dataset.csv'

In [4]:
embeddings_index = {}
with open('/home/tina/Downloads/glove.6B.100d.txt') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

In [5]:
def clean_dataframe(df):
    cleaned_df = df.copy()
    pattern = r'[^a-zA-Z\s]'
    cleaned_df['text'] = cleaned_df['text'].apply(lambda x: re.sub(pattern, '', x))
    
    return cleaned_df

In [6]:
def text_to_word_vectors(text, vector_space_dimension):    
    sentence_vectors = []
    for word in text.lower().split():
        if word in embeddings_index and len(sentence_vectors) < MAX_WORDS_IN_SENTENCE:
            sentence_vectors.append(embeddings_index[word])    
    if len(sentence_vectors) > 0:
        pad_len = vector_space_dimension-len(sentence_vectors)      
        if pad_len > 0:
            padding = [np.zeros_like(sentence_vectors[0])] * pad_len
            sentence_vectors = sentence_vectors + padding
    return sentence_vectors

In [7]:
dataset = pd.read_csv(CSV_PATH)
dataset = clean_dataframe(dataset)
texts = dataset['text'].astype(str)
dataset['vlen'] = [len(text_to_word_vectors(t, MAX_WORDS_IN_SENTENCE)) for t in texts]
dataset = dataset[dataset.vlen > 0]

texts = dataset['text'].astype(str)
labels = dataset['label']

In [8]:
data = [text_to_word_vectors(text, MAX_WORDS_IN_SENTENCE) for text in texts]
data = np.array(data)
targets = np.array(labels).astype("float32")

In [9]:
# data

In [15]:
test_x = data[:3000]
test_x = np.array(test_x).astype('float32')
test_x =  test_x.reshape(len(test_x), MAX_WORDS_IN_SENTENCE * WORD2VEC_DIMENSIONS)
test_y = targets[:3000]

train_x = data[3000:]
train_x = np.array(train_x).astype('float32')
train_x = train_x.reshape(len(train_x), MAX_WORDS_IN_SENTENCE * WORD2VEC_DIMENSIONS)
train_y = targets[3000:]

In [16]:
test_x.shape, test_y.shape

((3000, 50000), (3000,))

In [17]:
train_x.shape, train_y.shape

((2995, 50000), (2995,))

In [18]:
model = models.Sequential()
model.add(layers.Dense(50, activation="relu", input_shape=(MAX_WORDS_IN_SENTENCE * WORD2VEC_DIMENSIONS, )))  
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation="relu"))
model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation="relu"))
model.add(layers.Dense(1, activation="sigmoid")) 
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 50)                2500050   
                                                                 
 dropout_2 (Dropout)         (None, 50)                0         
                                                                 
 dense_5 (Dense)             (None, 50)                2550      
                                                                 
 dropout_3 (Dropout)         (None, 50)                0         
                                                                 
 dense_6 (Dense)             (None, 50)                2550      
                                                                 
 dense_7 (Dense)             (None, 1)                 51        
                                                                 
Total params: 2505201 (9.56 MB)
Trainable params: 2505

In [19]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
results = model.fit(train_x, train_y, epochs=5, batch_size=32, validation_data=(test_x, test_y))

Epoch 1/5


2023-09-15 09:02:44.957031: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 599000000 exceeds 10% of free system memory.


93/94 [============================>.] - ETA: 0s - loss: 0.7232 - accuracy: 0.5037

2023-09-15 09:02:48.384346: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 600000000 exceeds 10% of free system memory.


94/94 [==============================] - 4s 36ms/step - loss: 0.7231 - accuracy: 0.5035 - val_loss: 0.7027 - val_accuracy: 0.4983
Epoch 2/5
94/94 [==============================] - 3s 30ms/step - loss: 0.6863 - accuracy: 0.5452 - val_loss: 0.6900 - val_accuracy: 0.5463
Epoch 3/5
94/94 [==============================] - 3s 31ms/step - loss: 0.6568 - accuracy: 0.5753 - val_loss: 0.6685 - val_accuracy: 0.5970
Epoch 4/5
94/94 [==============================] - 3s 31ms/step - loss: 0.6151 - accuracy: 0.6531 - val_loss: 0.6629 - val_accuracy: 0.5980
Epoch 5/5
94/94 [==============================] - 3s 31ms/step - loss: 0.5228 - accuracy: 0.7205 - val_loss: 0.6548 - val_accuracy: 0.6287


In [20]:
model.save('./data/keras-review-100d.mdl')
loaded_model = models.load_model('./data/keras-review-100d.mdl')
scores = loaded_model.evaluate(test_x, test_y, verbose=0)
print("Loaded Model Accuracy: %.2f%%" % (scores[1] * 100))

INFO:tensorflow:Assets written to: ./data/keras-review-100d.mdl/assets


INFO:tensorflow:Assets written to: ./data/keras-review-100d.mdl/assets
2023-09-15 09:03:37.617369: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 600000000 exceeds 10% of free system memory.


Loaded Model Accuracy: 62.87%


In [21]:
scores = loaded_model.evaluate(test_x, test_y, verbose=0)
print("Loaded Model Accuracy: %.2f%%" % (scores[1] * 100))

2023-09-15 09:03:41.731378: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 600000000 exceeds 10% of free system memory.


Loaded Model Accuracy: 62.87%
